In [84]:
import pandas as pd
import numpy as np
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from scipy import stats

In [85]:
df = pd.read_csv(r"C:\Users\Bertold\Documents\CUNY\Fall 2020\Language Technology\lab1\data\ws353.tsv", sep='\t', names=["w1", "w2", "score"])
df

,w1,w2,score
0,tiger,cat,7.35
1,tiger,tiger,10.00
2,plane,car,5.77
3,train,car,6.31
4,television,radio,6.77
...,...,...,...
198,rooster,voyage,0.62
199,noon,string,0.54
200,chord,smile,0.54
201,professor,cucumber,0.31


In [86]:
# I dropped two rows that had differing parts of speech (one was noun, other was verb) 
#for their first result in the synsets:
df = df.drop([25], axis = 0)

In [87]:
df = df.drop([136], axis = 0)

In [88]:
#Making synset columns
df = df.join(df['w1'].apply(lambda word: pd.Series(wn.synsets(word))))
df.columns.values[3] = 'w1syn'
df1 = df[['w1', 'w2', 'score', 'w1syn']]
df2 = df1.join(df['w2'].apply(lambda word: pd.Series(wn.synsets(word))))
df2.columns.values[4] = 'w2syn'
df3 = df2[['w1', 'w2', 'score', 'w1syn', 'w2syn']]
df3

,w1,w2,score,w1syn,w2syn
0,tiger,cat,7.35,Synset('tiger.n.01'),Synset('cat.n.01')
1,tiger,tiger,10.00,Synset('tiger.n.01'),Synset('tiger.n.01')
2,plane,car,5.77,Synset('airplane.n.01'),Synset('car.n.01')
3,train,car,6.31,Synset('train.n.01'),Synset('car.n.01')
4,television,radio,6.77,Synset('television.n.01'),Synset('radio.n.01')
...,...,...,...,...,...
198,rooster,voyage,0.62,Synset('cock.n.04'),Synset('ocean_trip.n.01')
199,noon,string,0.54,Synset('noon.n.01'),Synset('string.n.01')
200,chord,smile,0.54,Synset('chord.n.01'),Synset('smile.n.01')
201,professor,cucumber,0.31,Synset('professor.n.01'),Synset('cucumber.n.01')


In [94]:
#Functions for the first three things:
def p_s(x, y):
    return wn.path_similarity(x, y)

def lch(x, y):
    return wn.lch_similarity(x, y)

def wus(x, y):
    return wn.wup_similarity(x, y)

#These need the IC (information content) thing. I used the Brown:
brown_ic = wordnet_ic.ic('ic-brown.dat')

def res(x, y, z):
    return wn.res_similarity(x, y, z)

def jcn(x, y, z):
    return wn.jcn_similarity(x, y, z)

def lin(x, y, z):
    return wn.lin_similarity(x, y, z)

In [95]:
#Testing path similarity:
print(p_s(wn.synset("cat.n.01"), wn.synset("tiger.n.01")))
print(p_s(df3['w1syn'][0], df3['w2syn'][0]))
#Testing the IC ones:
print(wn.synset('dog.n.01').res_similarity(wn.synset('cat.n.01'), brown_ic))
print(wn.synset('dog.n.01').jcn_similarity(wn.synset('cat.n.01'), brown_ic))
print(wn.synset('dog.n.01').lin_similarity(wn.synset('cat.n.01'), brown_ic))
print(wn.res_similarity(wn.synset('dog.n.01'), wn.synset('cat.n.01'), brown_ic))
print(res(wn.synset("dog.n.01"), wn.synset("cat.n.01"), brown_ic))

0.09090909090909091
0.09090909090909091
7.911666509036577
0.4497755285516739
0.8768009843733973
7.911666509036577
7.911666509036577


In [96]:
df3['path_similarity'] = df3.apply(lambda x: p_s(x['w1syn'], x['w2syn']), axis=1)
df3['wu_palm'] = df3.apply(lambda x: wus(x['w1syn'], x['w2syn']), axis=1)
# Had a part of speech problem with these 4:
df3['lea_cho'] = df3.apply(lambda x: lch(x['w1syn'], x['w2syn']), axis=1)
#NEEDS IC:
df3['resnik'] = df3.apply(lambda x: res(x['w1syn'], x['w2syn'], brown_ic), axis=1)
df3['jiang-con'] = df3.apply(lambda x: jcn(x['w1syn'], x['w2syn'], brown_ic), axis=1)
df3['lin'] = df3.apply(lambda x: lin(x['w1syn'], x['w2syn'], brown_ic), axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [97]:
# I took the first result from my synsets for each word -- parts of speech didn't match on two of rows, so I removed the rows
# I got this weird warning above; I looked into it, it said I should use .loc somewhere, but whatever, it worked
df3

,w1,w2,score,w1syn,w2syn,path_similarity,wu_palm,lea_cho,resnik,jiang-con,lin
0,tiger,cat,7.35,Synset('tiger.n.01'),Synset('cat.n.01'),0.090909,0.545455,1.239691,2.224150,5.930943e-02,2.087520e-01
1,tiger,tiger,10.00,Synset('tiger.n.01'),Synset('tiger.n.01'),1.000000,0.750000,3.637586,12.268375,1.000000e+300,1.000000e+00
2,plane,car,5.77,Synset('airplane.n.01'),Synset('car.n.01'),0.111111,0.666667,1.440362,5.921765,2.165083e-01,7.194339e-01
3,train,car,6.31,Synset('train.n.01'),Synset('car.n.01'),0.125000,0.666667,1.558145,5.768923,1.830526e-01,6.786666e-01
4,television,radio,6.77,Synset('television.n.01'),Synset('radio.n.01'),0.333333,0.900000,2.538974,8.924336,5.254700e-01,9.036511e-01
...,...,...,...,...,...,...,...,...,...,...,...
198,rooster,voyage,0.62,Synset('cock.n.04'),Synset('ocean_trip.n.01'),0.041667,0.080000,0.459532,-0.000000,4.334020e-02,-0.000000e+00
199,noon,string,0.54,Synset('noon.n.01'),Synset('string.n.01'),0.058824,0.111111,0.804373,-0.000000,4.544702e-02,-0.000000e+00
200,chord,smile,0.54,Synset('chord.n.01'),Synset('smile.n.01'),0.090909,0.285714,1.239691,0.596229,4.928296e-02,5.550591e-02
201,professor,cucumber,0.31,Synset('professor.n.01'),Synset('cucumber.n.01'),0.076923,0.500000,1.072637,2.224150,1.000000e-300,4.448301e-300


In [109]:
#Spearman:
def spear(x, y):
    return stats.spearmanr(x, y)

In [101]:
#Quick test:
print(spear([1, 2, 3], [4, 5, 6]))

SpearmanrResult(correlation=1.0, pvalue=0.0)


In [151]:
#I had to send these to numpy array things so I could calculate Spearman correlation:
scorearr = df3['score'].to_numpy()

psarr = df3['path_similarity'].to_numpy()
wuarr = df3['wu_palm'].to_numpy()
lcharr = df3['lea_cho'].to_numpy()
resarr = df3['resnik'].to_numpy()
jcnarr = df3['jiang-con'].to_numpy()
linarr = df3['lin'].to_numpy()

In [152]:
#Calculate:
p = spear(scorearr, psarr)
w = spear(scorearr, wuarr)
lc = spear(scorearr, lcharr)
r = spear(scorearr, resarr)
j = spear(scorearr, jcnarr)
l = spear(scorearr, linarr)

In [157]:
#Round and print:
print("Path Similarity: correlation: {:.4f}, pvalue:{:.4f}".format(p.correlation, p.pvalue))
print("Wu-Palmer: correlation: {:.4f}, pvalue:{:.4f}".format(w.correlation, w.pvalue))
print("Leacock-Chodorow: correlation: {:.4f}, pvalue:{:.4f}".format(lc.correlation, lc.pvalue))
print("Resnik: correlation: {:.4f}, pvalue:{:.4f}".format(r.correlation, r.pvalue))
print("Jiang-Conrath: correlation: {:.4f}, pvalue:{:.4f}".format(j.correlation, j.pvalue))
print("Lin: correlation: {:.4f}, pvalue:{:.4f}".format(l.correlation, l.pvalue))

Path Similarity: correlation: 0.4537, pvalue:0.0000
Wu-Palmer: correlation: 0.5457, pvalue:0.0000
Leacock-Chodorow: correlation: 0.4537, pvalue:0.0000
Resnik: correlation: 0.5827, pvalue:0.0000
Jiang-Conrath: correlation: 0.4841, pvalue:0.0000
Lin: correlation: 0.5693, pvalue:0.0000
